# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
output_data_file = "../WeatherPy/output_data/cities.csv"
with open(output_data_file, "r", encoding = "utf-8") as read_file:
    cities_df = pd.read_csv(read_file)
cities_df.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Ponta do Sol,32.67,-17.10,73.40,73,40,14.99,PT,1593288012
1,Busselton,-33.65,115.33,62.01,68,100,24.85,AU,1593287995
2,Saint George,37.10,-113.58,102.00,11,1,9.17,US,1593287787
3,Bang Saphan,11.21,99.51,77.43,87,100,4.99,TH,1593288013
4,Bredasdorp,-34.53,20.04,57.20,71,53,21.92,ZA,1593288013


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key = g_key)

locations = cities_df[["Lat", "Lng"]]
humidity = cities_df["Humidity"].astype(float)


In [4]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
vacation_city = cities_df.loc[(cities_df["Max Temp"] <= 80)&(cities_df["Max Temp"] >= 50)&
                              (cities_df["Humidity"] <= 40)&(cities_df["Cloudiness"] <= 30)&
                             (cities_df["Wind Speed"] < 15)]
print(len(vacation_city))
vacation_city


14


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
88,Leh,34.17,77.58,57.81,35,0,6.20,IN,1593288019
94,Khovd,48.01,91.64,66.45,32,1,4.76,MN,1593288019
119,Mount Isa,-20.73,139.50,53.60,37,0,4.70,AU,1593288021
158,East London,-33.02,27.91,68.23,27,2,14.83,ZA,1593288024
250,Karratha,-20.74,116.85,65.91,30,0,4.92,AU,1593287966
337,Diego de Almagro,-26.37,-70.05,75.40,19,10,13.18,CL,1593288037
371,Pozo Colorado,-23.49,-58.80,66.52,37,0,6.22,PY,1593288040
408,Kaeo,-35.10,173.78,51.01,3,1,5.01,NZ,1593288043
430,Luena,-11.78,19.92,57.43,37,0,3.06,AO,1593288044
450,Mokobeng,-23.00,27.67,54.25,37,0,3.49,BW,1593288046


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = vacation_city
hotel_names = []
hotel_df["Hotel Name"] = np.nan
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
params = {"type": "hotel", "key": g_key, "radius": 5000}


for index, row in hotel_df.iterrows():
    loc = f"{row['Lat']}, {row['Lng']}"
    params["location"] = loc

    response = requests.get(base_url, params=params).json()
    hotel_names.append(response['results'][1]['name'])
    
hotel_df["Hotel Name"] = hotel_names
hotel_df


C:\Users\osafi\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\osafi\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
88,Leh,34.17,77.58,57.81,35,0,6.20,IN,1593288019,Hotel Ladakh Greens
94,Khovd,48.01,91.64,66.45,32,1,4.76,MN,1593288019,Fortress of Hovd
119,Mount Isa,-20.73,139.50,53.60,37,0,4.70,AU,1593288021,ibis Styles Mt Isa Verona
158,East London,-33.02,27.91,68.23,27,2,14.83,ZA,1593288024,Tu Casa
250,Karratha,-20.74,116.85,65.91,30,0,4.92,AU,1593287966,ibis Styles Karratha
337,Diego de Almagro,-26.37,-70.05,75.40,19,10,13.18,CL,1593288037,hotel luna del desierto
371,Pozo Colorado,-23.49,-58.80,66.52,37,0,6.22,PY,1593288040,Planta de tratamiento de agua potable
408,Kaeo,-35.10,173.78,51.01,3,1,5.01,NZ,1593288043,Far North District Council
430,Luena,-11.78,19.92,57.43,37,0,3.06,AO,1593288044,La Paloma Restaurante
450,Mokobeng,-23.00,27.67,54.25,37,0,3.49,BW,1593288046,Mokobeng Primary School


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [8]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))